# COEN 140: Homework 4 (bonus) 
# Spam Classification with Logistic Regression
Robbie Culkin

Download the data at http://www.cse.scu.edu/~yfang/coen140/spambase.zip. The data is split into a training set (of size 3065) and a test set (of size 1536).

Consider the email spam data set. This consists of 4601 email messages, from which 57 features have been extracted. These are as follows:
* 48 features, in [0, 100], giving the percentage of words in a given message which match a given word on the list. The list contains words such as “business”, “free”, “george”, etc. (The data was collected by George Forman, so his name occurs quite a lot.)
* 6 features, in [0, 100], giving the percentage of characters in the email that match a given character on the list. The characters are ; ( [ ! $ #
* Feature 55: The average length of an uninterrupted sequence of capital letters (max is 40.3, mean is 4.9)
* Feature 56: The length of the longest uninterrupted sequence of capital letters (max is 45.0, mean is 52.6)
* Feature 57: The sum of the lengths of uninterrupted sequence of capital letters (max is 25.6, mean is 282.2)

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


# Read data

In [2]:
train = pd.read_csv('data/spambase/spam-train.csv',header=None)
test = pd.read_csv('data/spambase/spam-test.csv',header=None)

In [3]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.0,0.00,0.00,0.0,1.01,0.0,0.0,0.0,0.00,0.00,...,0.000,0.088,0.000,0.000,0.088,0.000,6.718,33.0,215.0,1.0
1,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,...,0.098,0.589,0.000,0.000,0.000,0.000,2.044,22.0,92.0,1.0
2,0.0,0.53,0.00,0.0,1.06,0.0,1.6,0.0,0.00,0.53,...,0.000,0.239,0.079,0.159,0.000,0.000,4.555,51.0,123.0,1.0
3,0.0,0.00,0.23,0.0,0.92,0.0,0.0,0.0,0.23,0.00,...,0.000,0.130,0.026,0.026,0.000,0.026,2.222,23.0,480.0,0.0
4,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,1.428,4.0,10.0,0.0


# X/y split

In [4]:
X_train = train.drop(57,axis=1)
y_train = train[:][57]

In [5]:
X_test = test.drop(57,axis=1)
y_test = test[:][57]

# Preprocessing
One can imagine performing several kinds of preprocessing to this data. Try each of the following separately:
## a. Standardize the columns so they all have mean 0 and unit variance.

In [6]:
def zmuv(X_in):
    X = X_in.copy()
    X = (X - X.mean(axis=0))/X.std(axis=0)
    return X

In [7]:
zmuv(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,-0.341197,-0.167892,-0.559543,-0.047041,0.990915,-0.358135,-0.279760,-0.274942,-0.318200,-0.395557,...,-0.114562,-0.160496,-0.219886,-0.164386,-0.449056,0.028504,-0.099663,0.054088,-0.129557,-0.114247
1,-0.341197,-0.167892,-0.559543,-0.047041,-0.450720,-0.358135,-0.279760,-0.274942,-0.318200,-0.395557,...,-0.114562,0.244309,2.059477,-0.164386,-0.449056,-0.297565,-0.099663,-0.091047,-0.215281,-0.306383
2,-0.341197,0.259688,-0.559543,-0.047041,1.062283,-0.358135,3.589996,-0.274942,-0.318200,0.460440,...,-0.114562,-0.160496,0.467108,0.603462,-0.169589,-0.297565,-0.099663,-0.013076,0.010717,-0.257958
3,-0.341197,-0.167892,-0.113615,-0.047041,0.862453,-0.358135,-0.279760,-0.274942,0.505942,-0.395557,...,-0.114562,-0.160496,-0.028801,0.088323,-0.403357,-0.297565,-0.037943,-0.085520,-0.207488,0.299705
4,-0.341197,-0.167892,-0.559543,-0.047041,-0.450720,-0.358135,-0.279760,-0.274942,-0.318200,-0.395557,...,-0.114562,-0.160496,-0.620253,-0.164386,-0.449056,-0.297565,-0.099663,-0.110175,-0.355555,-0.434474


## b. Transform the features using log(xij + 0.1).

In [8]:
def log_transform_helper(x):
    return log(x+0.1)

In [9]:
def log_transform(X_in):
    return X_in.applymap(log_transform_helper)

In [10]:
log_transform(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,-2.302585,-2.302585,-2.302585,-2.302585,0.104360,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,...,-2.302585,-2.302585,-1.671313,-2.302585,-2.302585,-1.671313,-2.302585,1.919566,3.499533,5.371103
1,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,...,-2.302585,-1.619488,-0.372514,-2.302585,-2.302585,-2.302585,-2.302585,0.762673,3.095578,4.522875
2,-2.302585,-0.462035,-2.302585,-2.302585,0.148420,-2.302585,0.530628,-2.302585,-2.302585,-0.462035,...,-2.302585,-2.302585,-1.081755,-1.720369,-1.350927,-2.302585,-2.302585,1.537942,3.933784,4.812997
3,-2.302585,-2.302585,-1.108663,-2.302585,0.019803,-2.302585,-2.302585,-2.302585,-1.108663,-2.302585,...,-2.302585,-2.302585,-1.469676,-2.071473,-2.071473,-2.302585,-2.071473,0.842429,3.139833,6.173994
4,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,...,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,-2.302585,0.423960,1.410987,2.312535


## c. Binarize the features using I(xij > 0).

In [11]:
def binarize_helper(x):
    if x > 0:
        return 1
    else:
        return 0

In [12]:
def binarize(X_in):
    return X_in.applymap(binarize_helper)

In [13]:
binarize(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,1
2,0,1,0,0,1,0,1,0,0,1,...,0,0,1,1,1,0,0,1,1,1
3,0,0,1,0,1,0,0,0,1,0,...,0,0,1,1,1,0,1,1,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1


# Logistic regression model, with gradient descent

In [14]:
def sigmoid(x):
    return 1./(1 + math.exp(-x))

v_sigmoid = np.vectorize(sigmoid)

In [15]:
class LogR_model:
    
    def fit(self,X_train,y_train,alpha):
        X = X_train.copy()
        X['bias'] = np.ones(X.shape[0])
        
        np.random.seed(42)
        w = np.random.normal(0,1,size=X.shape[1])
        
        epsilon = 1
        while epsilon > 10e-5:
            dLdw = np.dot((v_sigmoid(np.dot(X,w)) - y_train),X)
            
            w_new = w - alpha*dLdw
            epsilon = numpy.linalg.norm(w_new - w) #euclidian distance
            w = w_new
            
        self.w = w
        return self
    
    def predict(self,X_test):
        X = X_test.copy()
        X['bias'] = np.ones(X.shape[0])
        
        return v_sigmoid(np.dot(self.w,X.T)) # sigmoid to squash outputs to [0,1]

# Evaluate using different normalizations

In [16]:
def error(y_true,y_hat):
    incorrect = 0.
    for ii in range(len(y_true)):
        incorrect += abs(y_true[ii] - round(y_hat[ii]))
        
    return incorrect/len(y_true)

## a. Zero mean unit variance

In [17]:
lr = LogR_model()
lr.fit(zmuv(X_train),y_train,0.00001)

<__main__.LogR_model instance at 0x10bc81c20>

In [18]:
y_train_hat = lr.predict(zmuv(X_train))
print 'train error:', error(y_train,y_train_hat)

train error: 0.0711256117455


In [19]:
y_test_hat = lr.predict(zmuv(X_test))
print 'test error:', error(y_test,y_test_hat)

test error: 0.0690104166667


## b. Log transform

In [20]:
lr2 = LogR_model()
lr2.fit(log_transform(X_train),y_train,0.000001)

<__main__.LogR_model instance at 0x10bc56488>

In [21]:
y_train_hat = lr2.predict(log_transform(X_train))
print 'train error:', error(y_train,y_train_hat)

train error: 0.0688417618271


In [22]:
y_test_hat = lr2.predict(log_transform(X_test))
print 'test error:', error(y_test,y_test_hat)

test error: 0.05859375


## c. Binarize

In [23]:
lr3 = LogR_model()
lr3.fit(binarize(X_train),y_train,0.00001)

<__main__.LogR_model instance at 0x10bc56290>

In [24]:
y_train_hat = lr3.predict(binarize(X_train))
print 'train error:', error(y_train,y_train_hat)

train error: 0.068189233279


In [25]:
y_test_hat = lr3.predict(binarize(X_test))
print 'test error:', error(y_test,y_test_hat)

test error: 0.0638020833333
